In [1]:
import h2o
from h2o.automl import H2OAutoML
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.401-b10, mixed mode, sharing)
  Starting server from C:\Users\minhd\AppData\Roaming\Python\Python310\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\minhd\AppData\Local\Temp\tmp4dwwtx4y
  JVM stdout: C:\Users\minhd\AppData\Local\Temp\tmp4dwwtx4y\h2o_minhd_started_from_python.out
  JVM stderr: C:\Users\minhd\AppData\Local\Temp\tmp4dwwtx4y\h2o_minhd_started_from_python.err


C:\Users\minhd\AppData\Roaming\Python\Python310\site-packages\h2o\backend\server.py:385: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Bangkok
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 26 days
H2O_cluster_name:,H2O_from_python_minhd_2pp1n9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [3]:
train_df = pd.read_csv(r"train.csv", sep=';')
test_df = pd.read_csv(r"test.csv", sep=';')

In [4]:
type_dict = {cat:i for (i, cat) in enumerate(train_df['type'].unique())}
train_df['type']=train_df['type'].map(type_dict)
test_df['type']=test_df['type'].map(type_dict)

In [5]:
target = 'quality'
features = [c for c in train_df.columns if c not in target]

In [6]:
#train_df = train_df.drop_duplicates()

In [7]:
train_df_red = train_df[train_df['type']==type_dict['red']]
train_df_white = train_df[train_df['type']==type_dict['white']]
test_df_red = test_df[test_df['type']==type_dict['red']]
test_df_white = test_df[test_df['type']==type_dict['white']]

In [8]:
test_df_red.id

0      1257
5      5434
7      5146
12      219
13     6403
       ... 
804    5629
809    2007
811    2864
814    5465
817    5090
Name: id, Length: 233, dtype: int64

In [9]:
def stratify(df, seed=42):
    stratify = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    df['fold'] = np.zeros(len(df), dtype='int')
    
    for (i, (train_idx, val_idx)) in enumerate(stratify.split(df, y=df['quality'])):
        df['fold'].iloc[val_idx] = i
        
    return df

In [10]:
train_df_red = stratify(train_df_red, seed=30)
train_df_red

C:\Users\minhd\AppData\Local\Temp\ipykernel_23824\1914683325.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fold'] = np.zeros(len(df), dtype='int')
C:\Users\minhd\AppData\Local\Temp\ipykernel_23824\1914683325.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fold'].iloc[val_idx] = i


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,fold
1,7.7,0.5,0.26,1.9,0.062,9,31,0.9966,3.39,0.64,9.6,5,1,0
2,8.4,0.5,0.35,2.9,0.076,21,127,0.9976,3.23,0.63,9.2,5,1,2
18,8.8,0.4,0.49,2.8,0.083,18,111,0.9982,3.30,0.60,9.5,5,1,4
20,7.0,0.4,0.19,2.3,0.071,18,36,0.9948,3.39,0.56,10.9,5,1,0
21,8.2,0.4,0.38,2.6,0.093,23,60,0.9979,3.31,0.71,10.9,6,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6694,7.2,0.7,0.03,2.3,0.078,16,86,0.9974,3.53,0.57,9.7,5,1,3
6695,6.3,0.3,0.48,1.8,0.069,18,61,0.9959,3.44,0.78,10.3,6,1,0
6703,7.0,0.5,0.09,2.0,0.081,10,16,0.9948,3.43,0.59,11.5,6,1,0
6704,5.3,0.7,0.19,1.5,0.161,7,62,0.9940,3.62,0.61,11.0,5,1,2


In [11]:
train_df_white = stratify(train_df_white, seed=29)
train_df_white

C:\Users\minhd\AppData\Local\Temp\ipykernel_23824\1914683325.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fold'] = np.zeros(len(df), dtype='int')
C:\Users\minhd\AppData\Local\Temp\ipykernel_23824\1914683325.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fold'].iloc[val_idx] = i


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,fold
0,6.6,0.3,0.36,1.2,0.035,43,126,0.9909,3.01,0.63,11.4,6,0,2
3,7.5,0.4,0.33,5.0,0.045,30,131,0.9942,3.32,0.44,10.9,6,0,4
4,6.4,0.2,0.25,20.2,0.083,35,157,0.9998,3.17,0.50,9.1,5,0,1
5,7.6,0.3,0.52,13.2,0.042,61,148,0.9984,2.98,0.47,9.1,6,0,0
6,6.4,0.3,0.40,6.2,0.040,46,169,0.9953,3.15,0.46,9.3,6,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6709,7.2,0.2,0.19,7.7,0.045,53,176,0.9958,3.17,0.38,9.5,5,0,4
6710,6.7,0.3,0.34,7.5,0.036,39,124,0.9912,2.99,0.32,12.4,8,0,2
6711,6.6,0.3,0.24,3.3,0.034,29,99,0.9903,3.10,0.40,12.3,7,0,0
6712,8.0,0.2,0.31,5.6,0.049,24,97,0.9930,3.10,0.42,10.9,5,0,3


In [12]:
s = StandardScaler()

In [13]:
train_white = h2o.H2OFrame(train_df_white)
train_red = h2o.H2OFrame(train_df_red)

test_white = h2o.H2OFrame(test_df_white)
test_red = h2o.H2OFrame(test_df_red)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
target = 'quality'
features = [c for c in train_df.columns if c not in [target, 'type', 'fold']]
features

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [ ]:
aml = H2OAutoML(
    max_models=20, 
                seed=1, 
#                 max_runtime_secs = 60*60
               )
aml.train(x=features, y=target, training_frame=train_white,     fold_column='fold')

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows) 

In [ ]:
aml.leader.metalearner()

In [ ]:
aml_red = H2OAutoML(
    max_models=20, 
                seed=5, 
#                 max_runtime_secs = 60*60
               )
aml_red.train(x=features, y=target, training_frame=train_red, fold_column='fold')

In [ ]:
# View the AutoML Leaderboard
lb_red = aml_red.leaderboard
lb_red.head(rows=lb_red.nrows)  # 

In [ ]:
aml_red.leader.metalearner()

In [ ]:
test_y_red = aml_red.predict(test_red[features]).as_data_frame()['predict']
test_y_red

In [ ]:
test_y_white = aml.predict(test_white[features]).as_data_frame()['predict']
test_y_white

In [ ]:
test_df_red

In [ ]:
submission_df_red = pd.DataFrame({'id':test_df_red['id'].reset_index(drop=True), 'quality':test_y_red})

submission_df_red

In [ ]:
submission_df_white = pd.DataFrame({'id':test_df_white.id.reset_index(drop=True), 'quality':test_y_white})

submission_df_white

In [ ]:

submission_df = pd.concat([submission_df_red, submission_df_white])

In [ ]:
submission_df.to_csv('results9.csv', index=False)

In [ ]:
submission_df.value_counts(submission_df['quality'] > 7)